In [1]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [1]:
from src.main.models.bidirLSMT import BidirectionalLSTM
from src.main.utilities.utils import clean_text, stop_words_removal, get_dataset
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
 # define model and pipeline
bidirLSTM = BidirectionalLSTM(vocab_size=10000, embedding_dim=100, max_sequence_length=75, lstm_units=64)
bidirLSTM.pipeline = [clean_text, stop_words_removal]
# import dataset and apply pipeline
inputs, targets = get_dataset(one_hot=True)

inputs = bidirLSTM.run_pipeline(inputs, save=True)

# stratified train test split
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42, stratify=targets)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabriziodecastelli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fa

Pipeline execution time: 0:00:03.506384


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def tokenize(sequences):
    # Tokenization
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sequences)
    sequences_as_integers = tokenizer.texts_to_sequences(sequences)

    # Padding
    max_sequence_length = 100  # Choose an appropriate value based on your data
    return pad_sequences(sequences_as_integers, maxlen=max_sequence_length)

In [4]:
x_train = tokenize(x_train)
x_test = tokenize(x_test)

In [5]:
bidirLSTM.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, None, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,183,941 (4.52 MB)

 Trainable params: 1,183,941 (4.52 MB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
bidirLSTM.fit(x_train, y_train, batch_size=1000, epochs=1, validation_data=(x_test, y_test), verbose=1)

87/87 ━━━━━━━━━━━━━━━━━━━━ 60s 680ms/step - accuracy: 0.7011 - loss: 0.8527 - val_accuracy: 0.4034 - val_loss: 1.7013


In [10]:
%load_ext tensorboard
%tensorboard --logdir {config.TENSORBOARD_LOGS.format(repr(bidirLSTM))} --port=6969

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
bidirLSTM.evaluate(x_test, y_test)

677/677 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.3981 - loss: 1.7121


[1.7013485431671143, 0.403362900018692]

In [11]:
from src.main.utilities import plotting

y_pred = bidirLSTM.predict(x_test)
results = plotting.compute_performance(y_test, y_pred)
plotting.print_performance(results)


677/677 ━━━━━━━━━━━━━━━━━━━━ 23s 34ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets